# This Notebook will dive more into the Action class

***Disclaimer***: This file referenced some files in other directories. In order to have working cross referencing it's recommended to start the notebook server from the root directory (`Grid2Op`) of the package and __not__ in the `getting_started` sub directory:
```bash
cd Grid2Op
jupyter notebook
```

***NB*** For more information about how to use the package, a general help can be built locally (provided that sphinx is installed on the machine) with:
```bash
cd Grid2Op
make html
```
from the top directory of the package (usually `Grid2Op`).

Once build, the help can be access from [here](../documentation/html/index.html)

It is recommended to have a look at the [0_basic_functionalities](0_basic_functionalities.ipynb) and [1_Observation_Agents](1_Observation_Agents.ipynb) notebooks before getting into this one.

**Objective**

This notebook will cover the basic of how use at best the *Action* class to modify the powergrid efficiently. Indeed there are multiple concepts behind this class that are not very clear.

In [3]:
import os
import sys
import grid2op

In [4]:
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

## I) Action and Action Helper

2 main classes are usefull when dealing with *Action* in Grid2Op. The *Action* class is the most basic one. The *ActionHelper* is a tool that helps create and manipulate some actions.

As usual, it is recommended to have built locally the documentation, and to refer to the [action](../documentation/html/action.html) pages of the documentation or to the [Action.py](../grid2op/Action.py) file for a more detailed view on these two classes.

### IIA) Action Helper

**IMPORTANT NOTICE** Each *Agent* has its own 'action helper' attribute that can be called by `self.action_space`. This is the only recommended way to create a valid *Action*. Using its constructor is strongly NOT recommended, as it requires a deep knowledge of all the elements in the powergrid, as well as their names, their type, the order in which they are used in the backend etc. For performance reasons, no sanity check are performed to make sure the would be created action this way is compatible with the backend.

In the next cell, we retrieve the action space used by the Agent.

In [5]:
# import the usefull class
from grid2op.Runner import Runner
from grid2op.ChronicsHandler import Multifolder, GridStateFromFileWithForecasts
# make a runner
runner = Runner(init_grid_path=grid2op.CASE_14_FILE,
               path_chron=grid2op.CHRONICS_MLUTIEPISODE,
               gridStateclass=Multifolder,
               gridStateclass_kwargs={"gridvalueClass": GridStateFromFileWithForecasts},
               names_chronics_to_backend = grid2op.NAMES_CHRONICS_TO_BACKEND)
# initialize it
runner.make_env()
action_space = runner.agent.action_space

As opposed to the previous plateform, [pypownet](https://github.com/MarvinLer/pypownet), Grid2Op doesn't restrict anything on the type of Action. Generally speaking, an Action can modify production, loads, topology etc. By default though, an Action that an Agent can performed is a [TopologyAction](../documentation/html/action.html#grid2op.Action.TopologyAction), a specific type restricting it's usage to:

- change the status of the powerline (reconnect / disconnect) them
- change the way the objects (end of a powerline, generator or load) are interconnected at substations.

We will focus on this class in this notebook.

#### a) change vs set

To manipulate a powergrid, we decided to introduce two distinct (yet close) concepts that will affect the objects differently:

- **change** will:
  - connect a powerline if it was disconnected, or reconnect if it was connected
  - assign the object to the second bus if it was connected to bus 1 or assign it to bus 1 if it was connected to bus 2
- **set** will:
  - connect a powerline (regardless of its previous status) or disconnect it (regardless of its previous status)
  - assign the object to a specific bus (regardless of its previous bus and status -- for powerline)
  
This is another change compared to the previous pypownet implementation, were only the `change` concept was implemented. Having these two things helps understand what is really going on on the powergrid and allows to represent better the intention of the Agent, especially in debugging phase.

Of course, it is perfectly possible to use only the `change` capability and thus being closer to the original implementation.

Let's give a "concrete" example to highlight the difference between these two methods. Suppose we have a substation with 5 elements:
- the origin of powerline l_1
- the extremity of powerline l_2
- the extremity of powerline l_3
- a load c_1
- a generator g_1

Let's also assume the original configuration (before the action is applied, *ie* the configuration of the observation at time *t*) is:

| Object Name | Original Bus | Original Status|
|-------------|--------------|----------------|
| l_1         | 1            |    connected   |
| l_2         | 2            |    connected   |
| l_3         | NA           | disconnected   |
| c_1         | 1            | NA             |
| g_1         | 2            | NA             |

TO BE CONTINUED